In [1]:
!pip install demoji einops timm flash_attn openai-whisper datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 45.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 13.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
import re
import json as js
import os
import demoji
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
import copy
from datasets import Dataset
import whisper


%matplotlib inline

# Audio Model

Load the Whisper model "medium".

[Whisper paper ](https://arxiv.org/abs/2212.04356).

In [3]:
modelWhisp = whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 109MiB/s]


# Image Model

Load Florence-2-large model.

[Florence-2 Large paper ](https://arxiv.org/abs/2311.06242).

In [4]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Define Florence2 inference method

In [5]:
task_prompt = '<CAPTION>'

def run_example(task_prompt, imageToProcess, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=imageToProcess, return_tensors="pt")
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(imageToProcess.width, imageToProcess.height)
    )

    return generated_text

## Dataset Source

Place the dataset .zip file in the same directory and run the cell below.


In [8]:
!ls

sample_data


In [9]:
!unzip exported_chat.zip

Archive:  exported_chat.zip
   creating: exported_chat/
   creating: exported_chat/sample_whatsapp1/
  inflating: exported_chat/sample_whatsapp1/IMG_01.jpg  
 extracting: exported_chat/sample_whatsapp1/IMG_02.webp  
  inflating: exported_chat/sample_whatsapp1/sample_whatsapp.txt  
  inflating: exported_chat/sample_whatsapp1/summary.txt  
   creating: exported_chat/sample_whatsapp10/
  inflating: exported_chat/sample_whatsapp10/AUDIO_01.mp3  
  inflating: exported_chat/sample_whatsapp10/AUDIO_02.mp3  
  inflating: exported_chat/sample_whatsapp10/IMG_01.jpg  
  inflating: exported_chat/sample_whatsapp10/IMG_01.png  
  inflating: exported_chat/sample_whatsapp10/sample_whatsapp.txt  
  inflating: exported_chat/sample_whatsapp10/summary.txt  
   creating: exported_chat/sample_whatsapp11/
  inflating: exported_chat/sample_whatsapp11/sample_whatsapp.txt  
  inflating: exported_chat/sample_whatsapp11/summary.txt  
   creating: exported_chat/sample_whatsapp12/
  inflating: exported_chat/sample_

## Define Media Function

The following function converts media placeholders in the text into descriptions for images or transcriptions for audio.


In [16]:
def replace_image(match,file_path):
  start_tag = r'</s><s>'
  end_tag = r'</s>'
  img_name = match.group()
  pic_path = os.path.join(file_path, img_name)
  replacement_text = "Sent an image with this description:" + run_example(task_prompt, Image.open(pic_path)).replace("The image shows", "").strip()
  replacement_text = re.sub(start_tag, "", replacement_text)
  replacement_text = re.sub(end_tag, "", replacement_text)
  print(replacement_text)
  return replacement_text

In [15]:
def replace_audio(match, file_path):
  r_regex = r'</s><s>'
  audio_name = match.group()
  audio_path = os.path.join(file_path, audio_name)
  result = modelWhisp.transcribe(audio_path)

  print(result["text"])

  return result["text"].strip()

## Chat Parser

The following function uses various regex patterns to parse an entire WhatsApp txt file into a string. This string contains the complete dialogue between different parties and text representations of the media files.


In [17]:
# \d{1,2}/\d{1,2}/\d{2}, \d{1,2}:\d{2} (?:AM|PM) -
def clean_chat(file_name, keepEmojiText=False):
    text=""
    regex = r"\d{1,2}\/\d{1,2}\/\d{2}, \d{1,2}:\d{2}.(?:AM|PM) - "
    img_regex = r'\b\w+\.(?:jpg|jpeg|webp|png)\b'
    audio_regex = r'\b\w+\.(?:mp3|wav|opus)\b'

    with open(file_name, 'r', encoding="utf-8") as file:
        for line in file:
            if re.match(regex, line):
                line = re.sub(regex, "", line)
            matches = re.findall(img_regex, line)
            line = re.sub(img_regex, lambda match: replace_image(match, os.path.dirname(file_name)), line)
            line = re.sub(audio_regex, lambda match: replace_audio(match, os.path.dirname(file_name)), line)
            line = re.sub(r"\n", " ", line)
            if not keepEmojiText:
                line = demoji.replace(line)
            else:
                line = demoji.replace_with_desc(line, "|")
            text += line
    return text

print(clean_chat("./exported_chat/sample_whatsapp1/sample_whatsapp.txt", True))

Sent an image with this description:an aerial view of a beach with clear water
Sent an image with this description:A woman snorkeling in the ocean with a yellow flippers.
Alice: Hey everyone, have you thought about our summer vacation yet?  Bob: Yes! I was thinking about going to the beach. Here's a picture of the place I found. Sent an image with this description:an aerial view of a beach with clear water  Charlie: Wow, that looks amazing, Bob! I'm totally in. |smiling face with heart-eyes|  Dave: Looks great! I'm just concerned about the budget. How much would it cost us?  Alice: I did some research. It's about $500 each for a week, including accommodation.  Bob: That's not bad.  Charlie: That sounds perfect! I can already imagine us there. |relieved face||beach with umbrella|  Dave: I'm in too. Let's book it!  Alice: Awesome! I'll handle the booking.  Bob: Should we rent a car to explore the area?  Charlie: Definitely. I found a rental service that's affordable.   Dave: Nice! Let's 

## Chat Dataset

The following functions create a structured dataset from the previously parsed text and export it as a JSON string.


In [17]:
'''
create a dictionary with the following structure: id, text, golden_summary
'''
def create_dict(whatsapp_file, id, golden_summary_path):
    with open(golden_summary_path, 'r', encoding='utf-8') as file:
        summary = file.read()

    chat = {
        "id": id,
        "summary": summary,
        "dialogue": clean_chat(whatsapp_file)
    }
    return chat

In [19]:
path = "exported_chat"
dataset = []
for i, dir in enumerate(os.listdir(path)):
    chat_file_path = os.path.join(path, dir, "sample_whatsapp.txt")
    summary_file_path = os.path.join(path, dir, "summary.txt")
    print(chat_file_path, summary_file_path)
    if os.path.exists(chat_file_path) and os.path.exists(summary_file_path):
        dataset.append(create_dict(chat_file_path, i, summary_file_path))

dataset


exported_chat/sample_whatsapp7/sample_whatsapp.txt exported_chat/sample_whatsapp7/summary.txt
Sent an image with this description:A man with a tattoo on his arm giving a thumbs up.
Sent an image with this description:A large crowd of people watching a soccer game.
exported_chat/sample_whatsapp5/sample_whatsapp.txt exported_chat/sample_whatsapp5/summary.txt
Sent an image with this description:<s>A menu for a restaurant with a bowl of pasta on it.
Sent an image with this description:A restaurant with tables and chairs under a tree.
exported_chat/sample_whatsapp12/sample_whatsapp.txt exported_chat/sample_whatsapp12/summary.txt
Sent an image with this description:A large library with tables and chairs in it.
exported_chat/sample_whatsapp9/sample_whatsapp.txt exported_chat/sample_whatsapp9/summary.txt
exported_chat/sample_whatsapp3/sample_whatsapp.txt exported_chat/sample_whatsapp3/summary.txt
Sent an image with this description:A group of men playing soccer in front of a crowd.
Sent an ima

[{'id': 0,
  'summary': "The friends talks about the derby between Inter Milan and Milan, which ended with Inter Milan's victory. Alex, Sam and Jordan praised Inter Milan's performance, while Taylor and Casey highlighted the exceptional plays by Lautaro Martinez, considered man of the match, and Nicolo Barella.\nChris and Jamie, frustrated Milan fans, expressed their disappointment in Milan's performance. Casey and Taylor expressed optimism about Inter's form for the rest of the season, while Jamie stressed the need for Milan to improve their defense.\n",
  'dialogue': "Alex: Did you guys watch the derby? What a game! Sam: Yes! Inter Milan was on fire  Jordan: Totally! They really showed Milan who's boss. Taylor: Sent an image with this description:A man with a tattoo on his arm giving a thumbs up. That first goal by Lautaro Martinez was amazing! What a celebration tho Casey: Absolutely, Lautaro was unstoppable tonight. Alex: And that assist from Barella, so smooth! Sam: I think Milan'

In [20]:
import json

r = {'is_claimed': 'True', 'rating': 3.5}
json = json.dumps(dataset) # note i gave it a different name
print(json)

[{"id": 0, "summary": "The friends talks about the derby between Inter Milan and Milan, which ended with Inter Milan's victory. Alex, Sam and Jordan praised Inter Milan's performance, while Taylor and Casey highlighted the exceptional plays by Lautaro Martinez, considered man of the match, and Nicolo Barella.\nChris and Jamie, frustrated Milan fans, expressed their disappointment in Milan's performance. Casey and Taylor expressed optimism about Inter's form for the rest of the season, while Jamie stressed the need for Milan to improve their defense.\n", "dialogue": "Alex: Did you guys watch the derby? What a game! Sam: Yes! Inter Milan was on fire  Jordan: Totally! They really showed Milan who's boss. Taylor: Sent an image with this description:A man with a tattoo on his arm giving a thumbs up. That first goal by Lautaro Martinez was amazing! What a celebration tho Casey: Absolutely, Lautaro was unstoppable tonight. Alex: And that assist from Barella, so smooth! Sam: I think Milan's de